# Calculate VMT per household type

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from datetime import datetime
import replicaEVSE.sql_wrapper_functions as sql
import replicaEVSE.load_curve as sim
import replicaEVSE.vmt as vmt
from dask.distributed import Client, LocalCluster
%reload_ext autoreload
%autoreload 1

datadir = '../../data/'

pd.set_option('display.max_columns', None)

# cluster = LocalCluster(n_workers=63)  # Launches a scheduler and workers locally
# client = Client(cluster)  # Connect to distributed cluster and override default

### add home counties which will work for private_auto but not commercial which aren't in the population data table

In [2]:
df = pd.read_parquet(datadir+'/wa_pop_and_trips_sorted.parquet') # len = 51727268
counties = pd.read_parquet(datadir+'/population_counties_dataset.parquet', engine='pyarrow')

In [3]:
cdf = pd.merge(df, counties, on='person_id', how='left')
cdf.reset_index(drop=True)

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type,home_cty,home_st
0,18336634547462657450,10000002909730821040,PRIVATE_AUTO,SOCIAL,HOME,0 days 08:20:42,0 days 08:39:00,2.897196,nan,530250111002,47.098457,-119.319023,530250111003,47.117839,-119.303234,residential,single_family,residential,single_family,residential,single_family,residential,single_family,47.085370,-119.319090,47.118737,-119.298334,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA",Washington
1,1694659094182985471,10000002909730821040,PRIVATE_AUTO,HOME,SOCIAL,0 days 09:12:45,0 days 09:19:56,2.867602,nan,530250111003,47.117839,-119.303234,530250111002,47.098457,-119.319023,residential,single_family,residential,single_family,residential,single_family,residential,single_family,47.118737,-119.298334,47.085370,-119.319090,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,HOME,"Grant County, WA",Washington
2,7721623352602932778,10000002909730821040,ON_DEMAND_AUTO,SHOP,HOME,0 days 09:30:08,0 days 09:49:04,8.929170,nan,530250111002,47.098457,-119.319023,530250109022,47.152531,-119.293494,residential,single_family,residential,single_family,commercial,retail,commercial,retail,47.085370,-119.319090,47.144694,-119.278788,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA",Washington
3,3697919619727259534,10000002909730821040,CARPOOL,SHOP,SHOP,0 days 09:54:21,0 days 09:57:14,0.996870,nan,530250109022,47.152531,-119.293494,530250111001,47.123046,-119.285771,commercial,retail,commercial,retail,commercial,retail,commercial,retail,47.144694,-119.278788,47.131541,-119.276257,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA",Washington
4,9977271661694103189,10000002909730821040,WALKING,SHOP,SHOP,0 days 10:23:28,0 da

can see that not all entries have a home blockgroup

In [4]:
cdf.BLOCKGROUP.value_counts(dropna=False)

BLOCKGROUP
None            2156578
530330322103      53832
530630107001      49350
530730001003      48915
530530731251      45678
                 ...   
410390024011          1
410579604006          1
300310015002          1
300979670002          1
410710306012          1
Name: count, Length: 8132, dtype: int64

In [5]:
# change data types for consistency
cdf['home_cty'] = cdf['home_cty'].astype(str)
cdf['home_st'] = cdf['home_st'].astype(str)

## LDV VMT

In [13]:
weekday = 'thursday'
mode = 'PRIVATE_AUTO'
ldv_vmt_thur = vmt.calc_vmt_by_county(cdf, mode=mode, weekday=weekday) # .compute()
ldv_vmt_thur.to_csv(datadir+f'vmt_ldv_{weekday}.csv', index=False, mode='w')
ldv_vmt_thur

,home_cty,distance_miles
0,"Ada County, ID",3.440208e+04
1,"Adams County, ID",7.667243e+02
2,"Adams County, WA",5.665706e+05
3,"Asotin County, WA",2.993855e+05
4,"Baker County, OR",4.936120e+03
...,...,...
131,"Whitman County, WA",7.216444e+05
132,"Yakima County, WA",4.571530e+06
133,"Yamhill County, OR",2.987557e+04
134,"Yellowstone County, MT",2.733071e+02


In [14]:
weekday = 'saturday'

ldv_vmt_sat = vmt.calc_vmt_by_county(cdf, mode='PRIVATE_AUTO', weekday=weekday)
ldv_vmt_sat.to_csv(datadir+f'vmt_ldv_{weekday}.csv', index=False, mode='w')
ldv_vmt_sat.head()

,home_cty,distance_miles
0,"Ada County, ID",28044.363543
1,"Adams County, ID",32.248312
2,"Adams County, WA",563222.231352
3,"Asotin County, WA",211291.833937
4,"Baker County, OR",1575.514290


## HDV VMT

These don't have a home county associated with them so will have to use the desitnation blockgroup

In [6]:
# read in blockgroup info
bg_df = pd.read_csv(datadir+'blockgroup_counties.csv')
bg_df['destination_bgrp'] = bg_df.destination_bgrp.astype(str)
bg_df['destination_county'] = bg_df.County.astype(str)

In [7]:
bg_df

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,County,BLOCKGROUP,destination_bgrp,destination_county
0,53,29,971400,2,530299714002,Block Group 2,G5030,S,2814636,1797525,48.206277,-122.529653,"POLYGON ((-122.54657 48.208011, -122.542498 48...",Island County,530299714002,530299714002,Island County
1,53,29,971400,3,530299714003,Block Group 3,G5030,S,22645103,62368,48.212509,-122.499121,"POLYGON ((-122.527323 48.182455, -122.527269 4...",Island County,530299714003,530299714003,Island County
2,53,29,971500,1,530299715001,Block Group 1,G5030,S,14190551,3407670,48.252358,-122.456154,"POLYGON ((-122.485032 48.255072999999996, -122...",Island County,530299715001,530299715001,Island County
3,53,29,971500,2,530299715002,Block Group 2,G5030,S,13421381,9435655,48.214561,-122.452643,"POLYGON ((-122.481784 48.167791, -122.481692 4...",Island County,530299715002,530299715002,Island County
4,53,29,971600,1,530299716001,Block Group 1,G5030,S,13176547,2594118,48.171603,-122.513752,POLYGON ((-122.53916699999999 48.1904959999999...,Island County,530299716001,530299716001,Island County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5338,53,15,1100,2,530150011002,Block Group 2,G5030,S,311974,41091,46.148888,-122.909208,"POLYGON ((-122.91382899999999 46.14731, -122.9...",Cowlitz County,530150011002,530150011002,Cowlitz County
5339,53,15,1100,3,530150011003,Block Group 3,G5030,S,651187,0,46.139811,-122.902721,"POLYGON ((-122.911793 46.137749, -122.911517 4...",Cowlitz County,530150011003,530150011003,Cowlitz County
5340,53,15,1100,4,530150011004,Block Group 4,G5030,S,220828,0,46.138142,-122.912800,"POLYGON ((-122.91549 46.133441999999995, -122....",Cowlitz County,530150011004,530150011004,Cowlitz County
5341,53,15,1100,5,530150011005,Block Group 5,G5030,S,301290,0,46.136023,-122.908029,"POLYGON ((-122.91351 46.13303, -122.913243 46....",Cowlitz County,530150011005,530150011005,Cowlitz County


### there are two types of commercial traffic

In [8]:
cdf['vehicle_type'].value_counts()

vehicle_type
nan                  50274818
MEDIUM_COMMERCIAL     1335515
HEAVY_COMMERCIAL       116935
Name: count, dtype: int64

## Heavy duty VMT into destination county

In [9]:
weekday = 'thursday'
mode = 'COMMERCIAL'
vehicle_type = 'HEAVY_COMMERCIAL'
hdvs = cdf.loc[cdf['vehicle_type'] == vehicle_type]
# merge with the blockgroup county info
hdvs_cnty = pd.merge(hdvs, bg_df, on='destination_bgrp', how='left')
hdv_vmt = vmt.calc_vmt_by_county(hdvs_cnty, mode=mode, weekday=weekday, county_key='destination_county')
# hdv_vmt.to_csv(datadir+f'vmt_hdv_{vehicle_type}_{weekday}.csv', index=False)

In [11]:
weekday = 'saturday'
mode = 'COMMERCIAL'
vehicle_type = 'HEAVY_COMMERCIAL'
hdvs = cdf.loc[cdf['vehicle_type'] == vehicle_type]
# merge with the blockgroup county info
hdvs_cnty = pd.merge(hdvs, bg_df, on='destination_bgrp', how='left')
hdv_vmt = vmt.calc_vmt_by_county(hdvs_cnty, mode=mode, weekday=weekday, county_key='destination_county')
hdv_vmt.to_csv(datadir+f'vmt_hdv_{vehicle_type}_{weekday}.csv', index=False)

In [13]:
cdf['vehicle_type'].value_counts()

vehicle_type
nan                  50274818
MEDIUM_COMMERCIAL     1335515
HEAVY_COMMERCIAL       116935
Name: count, dtype: int64

## MDV VMT in destinatation county

In [14]:
weekday = 'saturday'
mode = 'COMMERCIAL'
vehicle_type = 'MEDIUM_COMMERCIAL'
mdvs = cdf.loc[cdf['vehicle_type'] == vehicle_type]
# merge with the blockgroup county info
mdvs_cnty = pd.merge(mdvs, bg_df, on='destination_bgrp', how='left')
mdv_vmt_sat = vmt.calc_vmt_by_county(mdvs_cnty, mode=mode, weekday=weekday, county_key='destination_county')
mdv_vmt_sat.to_csv(datadir+f'vmt_hdv_{vehicle_type}_{weekday}.csv', index=False)

## HDV VMT

These don't have a home county associated with them so will have to use the desitnation blockgroup

In [27]:
mdv_vmt_sat

vehicle_type
nan                  50274818
MEDIUM_COMMERCIAL     1335515
HEAVY_COMMERCIAL       116935
Name: count, dtype: int64

In [16]:
weekday = 'thursday'
mode = 'COMMERCIAL'
vehicle_type = 'MEDIUM_COMMERCIAL'
mdvs = cdf.loc[cdf['vehicle_type'] == vehicle_type]
# merge with the blockgroup county info
mdvs_cnty = pd.merge(mdvs, bg_df, on='destination_bgrp', how='left')
mdv_vmt_thu = vmt.calc_vmt_by_county(mdvs_cnty, mode=mode, weekday=weekday, county_key='destination_county')
mdv_vmt_thu.to_csv(datadir+f'vmt_hdv_{vehicle_type}_{weekday}.csv', index=False)

In [17]:
mdv_vmt_thu

,destination_county,distance_miles
0,Adams County,1.730667e+05
1,Asotin County,1.264568e+04
2,Benton County,3.309011e+05
3,Chelan County,1.352807e+05
4,Clallam County,1.015500e+05
5,Clark County,5.925547e+05
6,Columbia County,1.047632e+04
7,Cowlitz County,3.106777e+05
8,Douglas County,8.119115e+04
9,Ferry County,1.314956e+04


## BUSES?

I don't think there is anything we can do here...

# Everything after this is just dev testing code and can be ignored.

In [2]:
size = 100


users = np.random.randint(0, 1000, size=size).astype('int64')
trip = pd.DataFrame(
    {
        "user": users,
        "distance": np.random.randint(1, 101, size=size),
    }
)

c = np.array(["King", "Queen", "other"])
clist = np.random.choice(c, size=size)

pop = pd.DataFrame(
    {
        "user": users,
        "household_size": np.random.randint(1, 8, size=size),
        "county" : clist
    }
)

trip = dd.from_pandas(trip, npartitions=2)
pop = dd.from_pandas(pop, npartitions=2)
pop.sort_values('user')
trip.sort_values('user')
result = dd.merge(trip, pop, on="user", how="left")

single = result.loc[result['household_size'] == 1].groupby('county')['distance'].sum()
multi = result.loc[result['household_size'] != '1'].groupby('county')['distance'].sum()
vmt = single.compute()

In [3]:
multi.compute()

county
King     2071
Queen    1409
other    1449
Name: distance, dtype: int64

In [5]:
single.compute()

county
King     307
Queen    100
other    193
Name: distance, dtype: int64

In [24]:
result.head(10)

,user,distance,household_size,county
0,987,88,5,Queen
1,560,93,3,King
2,63,80,3,other
3,197,9,3,King
4,709,50,1,Queen
5,492,6,6,Queen
6,361,24,7,Queen
7,890,10,3,other
8,829,23,3,King
9,368,49,5,other


In [25]:
result.dtypes

user               int64
distance           int64
household_size     int64
county            object
dtype: object

In [5]:
vmt = single.compute()

: 

: 

In [26]:
out = single.compute()
out_multi = multi.compute()

In [27]:
out

county
King     190
Queen    771
other    129
Name: distance, dtype: int64

# Join the datasets

In [29]:
out_multi

county
King     1831
Queen    2127
other    1084
Name: distance, dtype: int64

In [167]:
trips_small = trips.head(10000)
persons_small = pop_ddf.head(10000)
trips_small = trips_small.sort_values(by=['person_id'], ascending=True)
persons_small = persons_small.sort_values(by=['person_id'], ascending=True)

In [168]:
len(persons_small['person_id'].unique()), len(trips_small['person_id'].unique())

(10000, 9994)

In [ ]:
for person in trips_small['person_id'].unique():
    print(person)
    trip_temp = trips_small.loc[trips_small['person_id']==person]
    person_temp = persons_small.loc[persons_small['person_id']==person]
    

In [162]:
persons_small.head()

,household_id,person_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh
7,12388187275325753555,10336420946307194711,410510082023,nan,nan,41051008202,nan,nan,35_64,53,M,white,not_hispanic_or_latino,lte_0,0.0,unemployed,some_college,not_attending_school,not_working,GQ,0,nan,not_working,GQ,same_house,1_person_group_quarters,lte_10000,0.0,GQ,GQ,GQ_structure,core,GQ_language,45.510140,-122.541693,NaN,NaN,NaN,NaN,unemployed
8,4789487632180387755,1080521673326832489,300490008003,300310007011,nan,30049000800,30031000701,nan,18_24,19,F,white,not_hispanic_or_latino,0_20000,182.0,employed,some_college,undergraduate,naics713940,GQ,0,nan,driving,GQ,within_us,1_person_group_quarters,lte_10000,0.0,GQ,GQ,GQ_structure,core,GQ_language,46.599123,-112.040589,45.680732,-111.026319,NaN,NaN,worked_from_home
3,18413256466912027068,11331589210799163497,530150021001,530150021001,nan,53015002100,53015002100,nan,18_24,20,F,white,hispanic_or_latino,100000_plus,233381.0,employed,some_college,not_attending_school,not_working,GQ,0,nan,walking,GQ,within_us,1_person_group_quarters,lte_10000,0.0,GQ,GQ,GQ_structure,core,GQ_language,46.128480,-122.936175,46.128287,-122.935595,NaN,NaN,worked_in_person
40,8648784058050036071,11333352192338053231,410670305011,410670305013,nan,41067030501,41067030501,nan,35_64,60,M,other_race_alone,hispanic_or_latino,20000_40000,23615.0,employed,no_school,not_attending_school,naics11,GQ,0,nan,worked_from_home,GQ,same_house,1_person_group_quarters,lte_10000,0.0,GQ,GQ,GQ_structure,core,GQ_language,45.463097,-122.782181,45.456414,-122.780823,NaN,NaN,worked_from_home
23,10567804410273856843,11578590798597194526,410510090002,nan,nan,41051009000,nan,nan,35_64,53,M,white,not_hispanic_or_latino,lte_0,0.0,unemployed,some_college,not_attending_school,not_working,GQ,0,nan,not_working,GQ,same_house,1_person_group_quarters,lte_10000,0.0,GQ,GQ,GQ_structure,core,GQ_language,45.488618,-122.533038,NaN,NaN,NaN,NaN,unemployed


In [157]:
result = pd.merge(trips_small, persons_small, on='person_id', how='left')

In [163]:
result

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh
0,16671747469368803178,10063917003561847179,CARPOOL,REGION_DEPARTURE,SHOP,0 days 19:26:00,0 days 19:39:15,6.303025,nan,080130133062,40.192236,-105.109522,080130132112,40.112775,-105.093314,commercial,retail,commercial,retail,nan,nan,nan,nan,40.190401,-105.102773,40.134308,-105.055378,saturday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14161940105906798253,10230136483887035220,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 09:55:16,0 days 13:16:49,208.460835,MEDIUM_COMMERCIAL,060490002001,41.735255,-121.352275,410191500001,43.151013,-123.373297,nan,nan,nan,nan,nan,nan,nan,nan,41.761580,-121.302100,43.152600,-123.373030,saturday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16774523842410535170,10488307126852964186,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 09:14:59,0 days 10:05:45,49.717741,MEDIUM_COMMERCIAL,080690001002,40.588401,-105.076229,560210014022,41.166119,-104.798958,nan,nan,nan,nan,nan,nan,nan,nan,40.584230,-105.079420,41.161590,-104.791030,saturday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15205166845100040833,10527497345335321861,PRIVATE_AUTO,REGION_DEPARTURE,SOCIAL,0 days 18:12:00,0 days 18:39:16,3.036239,nan,080130135061,40.195860,-105.083601,080130135081,40.177869,-105.065006,residential,single_family,residential,single_family,nan,nan,nan,nan,40.198496,-105.085051,40.177448,-105.055259,saturday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10496026919688790182,10570776030678321372,CARPOOL,REGION_DEPARTURE,MAINTENANCE,0 days 18:22:00,0 days 18:48:56,17.057504,nan,080590102063,39.834530,-105.088675,081230020072,40.037626,-105.049774,commercial,retail,commercial,retail,nan,nan,nan,nan,39.841010,-105.087130,40.037354,-105.055462,saturday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10994746225113822606,9175982185110204310,CARPOOL,REGION_DEPARTURE,SOCIAL,0 days 16:32:00,0 days 16:41:32,1.010879,nan,080690019033,40.317109,-105.349001,080690019033,40.317109,-105.349001,residential,single_family,residential,single_family,nan,nan,nan,nan,40.272991,-105.362009,40.264410,-105.349817,saturday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,2218127421787382015,9233634993516913290,C

In [154]:
9607977946022576238 > 9443392034007823004

True

In [118]:
trips_ddf.size.compute()

1448363504

In [119]:
len(trips_ddf)

51727268

In [137]:
# Take subset of trips using private autos as candidates for charging
trips_df = trips_ddf.loc[trips_ddf['mode'] == 'PRIVATE_AUTO']

# Create charge_type column from travel_purpose column
trips_df['charge_type'] = trips_df.travel_purpose

# Set charge_type to PUBLIC for all trips that are not HOME or WORK



In [126]:
trips_df['charge_type'].loc[~trips_df.charge_type.isin(
    ['WORK', 'HOME'])]

Dask Series Structure:
npartitions=566
    object
       ...
     ...  
       ...
       ...
Name: charge_type, dtype: object
Dask Name: loc-series, 19 graph layers

In [133]:
trips_df['charge_type'] = trips_df.charge_type.where(~trips_df.charge_type.isin(
    ['WORK', 'HOME']), 'PUBLIC')

In [136]:
trips_df.charge_type.isin(['WORK', 'HOME']).head()

Dask Series Structure:
npartitions=566
    bool
     ...
    ... 
     ...
     ...
Name: charge_type, dtype: bool
Dask Name: isin, 29 graph layers

In [134]:
trips_df['charge_type'].value_counts().compute()

SHOP                   6309339
EAT                    3298151
SOCIAL                 2209488
RECREATION             1142539
MAINTENANCE            1082962
OTHER_ACTIVITY_TYPE     443959
SCHOOL                  310159
WORK_FROM_HOME          265234
LODGING                  60404
REGION_DEPARTURE          4300
Name: charge_type, dtype: int64

In [128]:
trips_df['charge_type'].value_counts().compute()

HOME                   9325805
SHOP                   6309339
EAT                    3298151
WORK                   2672120
SOCIAL                 2209488
RECREATION             1142539
MAINTENANCE            1082962
OTHER_ACTIVITY_TYPE     443959
SCHOOL                  310159
WORK_FROM_HOME          265234
LODGING                  60404
REGION_DEPARTURE          4300
Name: charge_type, dtype: int64

In [110]:
simulation_id = 'test'
sim_output = sim.simulate_person_load(
    pop_ddf[:1000],
    trips_ddf[:1000],
    existing_load,
    simulation_id,
    managed=False
)

NotImplementedError: 'DataFrame.iloc' only supports selecting columns. It must be used like 'df.iloc[:, column_indexer]'.

In [ ]:
# run the simulations
print('Running simulations')
now = datetime.now()
print('Start time: ', now.strftime("%H:%M:%S"))
sim.run_charge_simulation(
        pop_ddf,
        trips_ddf,
        
        database_pw = pw,
        person_columns = person_columns,
        trips_columns = trips_columns,
        existing_load = existing_load,
        simulation_id = 'test',
        managed = False
    )

In [88]:
# join the pop and trip dataframes on person_id

df1 = pd.DataFrame(
    {
        "person_id": ["1", "2", "3", "4", "5"],
        "distance": ["B0", "B1", "B2", "B3", "B4"],

    }
)


df2 = pd.DataFrame(
    {
        "person_id": ["1", "2", "3", "4"],
        "house": ["B4", "B5", "B6", "B7"],
    }
)


In [93]:
result = pd.merge(df1, df2, on="person_id", how="outer")

In [94]:
result

,person_id,distance,house
0,1,B0,B4
1,2,B1,B5
2,3,B2,B6
3,4,B3,B7
4,5,B4,NaN


In [6]:
person_columns = [
    'household_id', 'person_id', 'BLOCKGROUP', 'BLOCKGROUP_work', 'age',
    'sex', 'race', 'ethnicity', 'individual_income', 'employment', 'education',
    'industry', 'household_role', 'commute_mode', 'household_size', 'household_income',
    'family_structure', 'vehicles', 'building_type', 'resident_type'
]
trips_columns = [
    'activity_id', 'person_id', 'distance_miles', 'mode', 'travel_purpose', 'start_time',
    'end_time', 'origin_bgrp', 'destination_bgrp', 'origin_land_use_l1', 'destination_land_use_l1',
    'origin_building_use_l1', 'destination_building_use_l1', 'vehicle_type','weekday'
]
existing_load = pd.read_csv('../../data/EIA_demand_summary.csv') 
#Created in the EIA_data_download.ipynb notebook

In [7]:
# drop the charge tables from previous runs
sql.execute_query(connection_replica, 'DROP TABLE IF EXISTS loads;')
sql.execute_query(connection_replica, 'DROP TABLE IF EXISTS charges;')
sql.execute_query(connection_replica, 'DROP TABLE IF EXISTS simulation_runs;')

create_charges_table="""
CREATE TABLE charges (
  charge_id VARCHAR(24) PRIMARY KEY,
  activity_id VARCHAR(20),
  simulation_id VARCHAR(4),
  charger_power_kW DECIMAL(5,1),
  charge_energy_used_kWh DECIMAL(4,1),
  charge_opportunity_remaining_kWh DECIMAL(5,1)
  );
"""
create_load_table="""
CREATE TABLE loads (
  load_segment_id VARCHAR(26) PRIMARY KEY,
  charge_id VARCHAR(24),
  window_start_time TIME,
  window_end_time TIME,
  load_kW DECIMAL(5,1)
  );
"""
create_simulation_table="""
CREATE TABLE simulation_runs (
  simulation_id VARCHAR(4) PRIMARY KEY,
  simulation_date DATETIME,
  simulation_attributes VARCHAR(200)
  );
"""

sql.execute_query(connection_replica, create_charges_table) # Execute our defined query
sql.execute_query(connection_replica, create_load_table) # Execute our defined query
sql.execute_query(connection_replica, create_simulation_table) # Execute our defined query

In [8]:
# run the simulations
print('Running simulations')
now = datetime.now()
print('Start time: ', now.strftime("%H:%M:%S"))
sim.run_charge_simulation(
        person_ids = new_chunked_list[0][0:10],
        database_name = 'replica',
        database_pw = pw,
        person_columns = person_columns,
        trips_columns = trips_columns,
        existing_load = existing_load,
        simulation_id = 'test',
        managed = False
    )

Running simulations
Start time:  14:04:16
MySQL Database connection successful
('9089435034280675508',)
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '), ('8607650910989625790',), ('17138200088452299858',), ('10060598750073083047',' at line 3'


''

In [27]:
person_ids = chunked_list[0][0:10]

In [28]:
str(tuple(person_ids))

"(('11992089757092215464',), ('8607650910989625790',), ('17138200088452299858',), ('10060598750073083047',), ('18259766398818683053',), ('12632939365244839467',), ('3915955982574925961',), ('3613240232523506087',), ('16415562233222284884',), ('9089435034280675508',))"

In [ ]:
new_chunked_list = []
for chunk in chunked_list:
    new_chunked_list.append([str(x).strip("),").strip("(") for x in chunk])

In [50]:
person_ids = new_chunked_list[0]
q1 = """
    SELECT """+str(trips_columns).replace('[', '').replace(']', '').replace('\'', '')+"""
    FROM trips
    WHERE person_id IN """+str(tuple(person_ids))+""";
    """
results_trips = sql.read_query(connection_replica, q1)


[]